In [1]:
import numpy as np
import pandas as pd

# load dataset
df = pd.read_csv("/content/finalencodeddsfr.csv")

# keep only demographic one-hot columns
context_cols = [col for col in df.columns if col.startswith("marital_status_")
                or col.startswith("age_range_")
                or col.startswith("family_size_")
                or col.startswith("income_bracket_")]

X = df[context_cols].values
A = df["coupon_id"].values
R = df["reward"].values

# initialize LinUCB memory
d = X.shape[1]
alpha = 0.1
actions = np.unique(A)
A_matrices = {a: np.eye(d) for a in actions}
b_vectors = {a: np.zeros(d) for a in actions}

# training loop
for i in range(len(X)):
    x_t = X[i]
    a_t = A[i]
    r_t = R[i]
    A_matrices[a_t] += np.outer(x_t, x_t)
    b_vectors[a_t] += r_t * x_t

# compute theta for each coupon
theta = {a: np.linalg.inv(A_matrices[a]).dot(b_vectors[a]) for a in actions}

# turn one-hot rows into readable profile strings
def profile_from_row(row):
    active = [col for col in context_cols if row[col] == 1]
    return "|".join(active) if active else "Unknown"

df["profile"] = df[context_cols].apply(profile_from_row, axis=1)

# group by profile
grouped_profiles = df.groupby("profile")

recommendations = []
for profile, group in grouped_profiles:
    x_vector = group[context_cols].iloc[0].values  # representative vector
    scores = {a: theta[a].dot(x_vector) for a in actions}
    best_coupon = max(scores.items(), key=lambda x: x[1])
    recommendations.append((profile, best_coupon[0], best_coupon[1], len(group)))

for rec in recommendations:
    print(f"Profile: {rec[0]} | Group size: {rec[3]} → Best coupon {rec[1]} with predicted value {rec[2]:.2f}")

Profile: marital_status_Married|age_range_18-25|family_size_2|income_bracket_11.0 | Group size: 2 → Best coupon 1075 with predicted value 2.09
Profile: marital_status_Married|age_range_18-25|family_size_2|income_bracket_2.0 | Group size: 3 → Best coupon 529 with predicted value 2.22
Profile: marital_status_Married|age_range_18-25|family_size_2|income_bracket_4.0 | Group size: 7 → Best coupon 1075 with predicted value 2.19
Profile: marital_status_Married|age_range_18-25|family_size_3|income_bracket_2.0 | Group size: 2 → Best coupon 982 with predicted value 2.06
Profile: marital_status_Married|age_range_18-25|family_size_3|income_bracket_4.0 | Group size: 4 → Best coupon 619 with predicted value 1.95
Profile: marital_status_Married|age_range_18-25|family_size_3|income_bracket_5.0 | Group size: 47 → Best coupon 870 with predicted value 2.17
Profile: marital_status_Married|age_range_18-25|family_size_3|income_bracket_6.0 | Group size: 2 → Best coupon 597 with predicted value 2.10
Profile: 